In [ ]:
# Copyright 2025 Forusone

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Agent Development Kit


### Install adk

In [1]:
%pip install --upgrade --quiet google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.7 MB/s eta 0:00:00


In [2]:
# @title Project Information

PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"

MODEL_LIST = ["gemini-2.0-flash", \
              "gemini-2.5-pro-preview-03-25", \
              "gemini-2.5-flash-preview-04-17"]

MODEL_NAME = MODEL_LIST[1]
print(MODEL_NAME)

gemini-2.5-pro-preview-03-25


In [3]:
# @title Authentication to access to GCP
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)

## Agent on adk

In [4]:
#@title Build a Agent with adk.
from google.adk.agents import Agent
from google.adk.tools import google_search

agent = Agent(
    name="search_assistant",
    model=MODEL_NAME,
    instruction="You are a helpful assistant. Answer user questions using Google Search when needed.",
    description="An assistant that can search the web.",
    tools=[google_search] # Use Google search tool
)


In [5]:
#@title Integrate an agent with Agent Engine

from vertexai.preview.reasoning_engines import AdkApp

app = AdkApp(agent=agent)

for event in app.stream_query(
   user_id="shins",  # Required
   message="What is the exchange rate from US dollars to Korean currency today ?",
):
   print(event)
   print(event['content']['parts'][0]['text'])

{'content': {'parts': [{'text': 'Based on the search results for today, Saturday, April 26, 2025, here are the current exchange rates found for US Dollars (USD) to South Korean Won (KRW):\n\n*   **XE.com:** 1 USD = 1,437.42 KRW\n*   **Forbes Advisor (as of 10:05 UTC):** 1 USD = 1,438.36 KRW\n*   **Wise:** 1 USD = 1,438.69 KRW\n*   **Revolut:** Offers rates around 1 USD = 1,428.77 KRW to 1 USD = 1,439.41 KRW (rates may vary in-app and fees can apply). One source specifically stated 1 USD = 1,437.44 KRW.\n*   **The Economic Times:** 1 USD = 1,441.7 KRW\n\n**In summary:**\n\nAs of today, April 26, 2025, 1 US Dollar is equivalent to approximately **1,437 to 1,442 South Korean Won (KRW)**.\n\nPlease note that exchange rates fluctuate constantly and can vary slightly depending on the specific provider (like banks, currency exchange services, or money transfer platforms like Wise, Revolut, OFX, or Western Union). The exact rate you get will depend on when and where you perform the exchange, a

## Deploy agent on Agent Engine for a managed service.

In [7]:
#@title Create a bucket to store agent engine artifacts

BUCKET_URI = f"gs://agent-0417"
!gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://agent-0417/...
ServiceException: 409 A Cloud Storage bucket named 'agent-0417' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [8]:
#@title Initialize Vertex AI with Staging Bucket.
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

In [9]:
# @title Deploy your agent on Vertex AI

from vertexai import agent_engines

# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.preview.reasoning_engines.ReasoningEngine#vertexai_preview_reasoning_engines_ReasoningEngine_create

remote_agent = agent_engines.create(
    app,
    display_name="currency agent with adk",
    gcs_dir_name = "ai-agent-adk",
    description="This is a simple agent with adk on agent engine.",
    requirements=[
        "google-adk",
        "cloudpickle==3.0",
    ],
    extra_packages = []
)

INFO:vertexai.agent_engines:Identified the following requirements: {'google-cloud-aiplatform': '1.90.0', 'cloudpickle': '3.1.1', 'pydantic': '2.11.3'}
INFO:vertexai.agent_engines:The following requirements are appended: {'pydantic==2.11.3'}
INFO:vertexai.agent_engines:The final list of requirements: ['google-adk', 'cloudpickle==3.0', 'pydantic==2.11.3']
INFO:vertexai.agent_engines:Using bucket agent-0417
INFO:vertexai.agent_engines:Wrote to gs://agent-0417/ai-agent-adk/agent_engine.pkl
INFO:vertexai.agent_engines:Writing to gs://agent-0417/ai-agent-adk/requirements.txt
INFO:vertexai.agent_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.agent_engines:Writing to gs://agent-0417/ai-agent-adk/dependencies.tar.gz
INFO:vertexai.agent_engines:Creating AgentEngine
INFO:vertexai.agent_engines:Create AgentEngine backing LRO: projects/721521243942/locations/us-central1/reasoningEngines/2135700181885124608/operations/5503555528132919296
INFO:vertexai.agent_engines:View progress 

In [10]:
# @title Query from remote engine.

for event in remote_agent.stream_query(
    user_id="shins",
    message="What is the exchange rate from US dollars to Korean Won currency on today?",
):
    print(event)

{'content': {'parts': [{'text': 'As of Saturday, April 26, 2025 at 2:57 PM UTC, here are the current exchange rates found for US Dollars (USD) to South Korean Won (KRW):\n\n*   **XE.com:** 1 USD = 1,437.42 KRW\n*   **Revolut (Australia):** 1 USD = 1,439.41 KRW (approximately, rates may vary and fees may apply)\n*   **Currency.ME.UK:** 1 USD = 1,438.36 KRW (as of 02:00 on April 26, 2025)\n*   **Wise:** 1 USD = 1,438.69 KRW\n*   **Investing.com:** Ask rate: 1 USD = 1,439.49 KRW\n\nPlease note that exchange rates fluctuate constantly. The rate you get when actually converting money may differ slightly depending on the provider and any associated fees.'}], 'role': 'model'}, 'grounding_metadata': {'grounding_chunks': [{'web': {'domain': 'wise.com', 'title': 'wise.com', 'uri': 'https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAKcMl4qDl4vTk1Vup1mM8XZ_SO2slPXmaFXzBBLA8-_DkcMcfAlMmUhPfGlmXV1W9dOA4rk4jAWeiOqGa6Jc3WBvz52agEG4wXUkNMz84r2VGHgyLZRfN8msgXO3Bqtm5pd8LCXcZYpEACDd-coE4

In [11]:
#@title Download common functions and execute.
!wget https://raw.githubusercontent.com/shins777/ai_agent/main/common/common_agent_engine.ipynb

# Execute common function for agent engin.
%run "common_agent_engine.ipynb"

--2025-04-26 14:57:41--  https://raw.githubusercontent.com/shins777/ai_agent/main/common/common_agent_engine.ipynb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2046 (2.0K) [text/plain]
Saving to: ‘common_agent_engine.ipynb’

common_agent_engine 100%[===================>]   2.00K  --.-KB/s    in 0s      

2025-04-26 14:57:41 (28.0 MB/s) - ‘common_agent_engine.ipynb’ saved [2046/2046]

Function get_agent_engine(display_name:str) added
Function show_agents() added
Function delete_agent(name:str)


## Agent Management

In [23]:
#@title Show agent list
show_agents()

Agent 0: 
	Display Name [currency agent with adk] 
	Name [2135700181885124608] 
	Creation Time [2025-04-26 14:53:02.246908+00:00] 
	Resource Name [projects/721521243942/locations/us-central1/reasoningEngines/2135700181885124608]

Agent 1: 
	Display Name [currency agent with adk] 
	Name [8720807262030921728] 
	Creation Time [2025-04-18 08:10:36.636926+00:00] 
	Resource Name [projects/721521243942/locations/us-central1/reasoningEngines/8720807262030921728]



In [14]:
#@title Get a agent with display name.
agent = get_agent_engine(display_name="currency agent with adk")
agent

resource name: projects/ai-hangsik/locations/us-central1/reasoningEngines/2135700181885124608

In [15]:
# @title Query from remote engine.
for event in agent.stream_query(
    user_id="shins",
    message="What is the exchange rate from US dollars to Korean Won currency on toady?",
):
    print(event)

{'content': {'parts': [{'text': 'Based on the search results for the US Dollar (USD) to South Korean Won (KRW) exchange rate on April 14, 2025:\n\n*   Several sources indicate the exchange rate for that day.\n*   **Currency.ME.UK** and **Foreign Exchange UK** reported 1 USD = 1,420.4934 KRW.\n*   **Pound Sterling LIVE** provided daily rates: Open 1,420.4000, Close 1,421.3300, High 1,431.7400, Low 1,414.5100, Mid 1,423.1250 KRW per USD.\n*   **Investing.com** reported: Open 1,420.41, High 1,431.99, Low 1,419.20, and a closing price of 1,426.63 KRW per USD.\n*   **ExchangeRates.org.uk** showed a closing rate of 1 USD = 1420 KRW, with a high of 1422 and a low of 1421 KRW (Note: This source seems to round the values).\n*   **YCharts**, sourcing from the European Central Bank, provided a rate of 0.0007 USD per 1 KRW for April 14, 2025. This translates to approximately 1 USD = 1428.57 KRW (1 / 0.0007).\n\nPlease note that different financial data providers may report slightly different figur

In [22]:
#@title Delete a agent with name.
delete_agent(name="5442468208281911296")

INFO:vertexai.agent_engines:Delete Agent Engine backing LRO: projects/721521243942/locations/us-central1/operations/2962399438389116928
INFO:vertexai.agent_engines:Agent Engine deleted. Resource name: projects/ai-hangsik/locations/us-central1/reasoningEngines/5442468208281911296


Deleted 5442468208281911296


## End of notebook